In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms 

## Get Device for Training

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


## Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Don't call `forward` directly, instead pass data to the model to start training

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2])


## Model Layers

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
# flatten 28x28 image into contiguous array of 784 pixel values
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
# apply linear transformation in input using stored weights and biases
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# apply non-linear activations to help learn wide variety of phenomena
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0807, -0.2861,  0.4267,  0.0752, -0.1389, -0.2902,  0.0176,  0.2851,
         -0.4047, -0.5002, -0.7949, -0.0422,  0.0154,  0.6567,  0.2234, -0.2188,
          0.2474, -0.1077,  0.1532,  0.1898],
        [-0.3151, -0.2432,  0.3739, -0.2953, -0.0975, -0.3010, -0.0798,  0.0161,
         -0.4126, -0.2083, -0.4384, -0.0688,  0.2240,  0.2856,  0.1602, -0.2257,
          0.4575,  0.1506,  0.3748, -0.0818],
        [-0.1174, -0.0556, -0.0061, -0.2654, -0.1290, -0.2919, -0.2150,  0.1058,
         -0.4650, -0.3314, -0.6730, -0.1780,  0.1292,  0.7194,  0.1582,  0.0108,
          0.6134,  0.0506,  0.2540, -0.0426]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4267, 0.0752, 0.0000, 0.0000, 0.0176, 0.2851, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0154, 0.6567, 0.2234, 0.0000, 0.2474, 0.0000,
         0.1532, 0.1898],
        [0.0000, 0.0000, 0.3739, 0.0000, 0.0000, 0.0000, 0.0000, 0.0161, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2240, 0.2856, 0.16

In [12]:
# nn.Sequential can stack multiple layers together
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.0119, -0.1613,  0.1135,  0.1707,  0.0138, -0.0986, -0.0256,  0.0091,
          0.2024, -0.2500],
        [ 0.0978, -0.1685,  0.2090,  0.1162,  0.0510, -0.1667, -0.0195, -0.0155,
          0.0873, -0.2424],
        [ 0.1254, -0.1465,  0.1998,  0.1254,  0.0434, -0.1608, -0.0469, -0.0523,
          0.1269, -0.2517]], grad_fn=<AddmmBackward0>)


In [14]:
# last linear layer returns logits, pass to Softmax for output
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1004, 0.0845, 0.1112, 0.1177, 0.1006, 0.0899, 0.0967, 0.1001, 0.1215,
         0.0773],
        [0.1098, 0.0841, 0.1227, 0.1118, 0.1048, 0.0843, 0.0976, 0.0980, 0.1087,
         0.0781],
        [0.1126, 0.0858, 0.1213, 0.1126, 0.1038, 0.0846, 0.0948, 0.0943, 0.1128,
         0.0773]], grad_fn=<SoftmaxBackward0>)


## Model Parameters

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0094, -0.0010, -0.0242,  ...,  0.0329, -0.0216,  0.0345],
        [ 0.0069, -0.0179, -0.0153,  ..., -0.0276,  0.0175,  0.0082]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0290, -0.0025], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0031,  0.0199,  0.0026,  ...,  0.0124, -0.0138,  0.0436],
        [-0.0385,  0.0054, -0.0008,  ...,  0.0060,  0.0156,  0.0209]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Siz